In [1]:

import os
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader
from pathlib import Path
from langchain.text_splitter import RecursiveCharacterTextSplitter

from pinecone import ServerlessSpec
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA

c:\Users\Kavitha\StudioProjects\bruno_app\backend\bruno_server_venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pinecone_apikey=os.getenv("PINECONE_API_KEY")
groq_apikey=os.getenv("GROQ_API_KEY")
pinecone_environment=os.getenv("PINECONE_ENVIRONMENT")

In [3]:
embeddings = HuggingFaceEmbeddings()

C:\Users\Kavitha\AppData\Local\Temp\ipykernel_3564\3655315981.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\Kavitha\AppData\Local\Temp\ipykernel_3564\3655315981.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [17]:
from pinecone import Pinecone
pc = Pinecone(pinecone_apikey)


In [6]:

cloud ='aws'
region =pinecone_environment

spec = ServerlessSpec(cloud=cloud, region=region)

In [9]:
def extract_and_vectorize(pc,spec,embeddings):
    UPLOAD_FOLDER = Path("uploaded_files")
    all_text = ""

    for file_path in UPLOAD_FOLDER.glob("*"):
        if file_path.suffix == ".pdf":
            loader = PyPDFLoader(str(file_path))
        elif file_path.suffix in [".doc", ".docx"]:
            loader = Docx2txtLoader(str(file_path))
        elif file_path.suffix == ".txt":
            loader = TextLoader(str(file_path))
        else:
            continue

        documents = loader.load()
        all_text += " ".join([doc.page_content for doc in documents])

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text(all_text)

    index_name = "bruno"

    if index_name not in pc.list_indexes().names():
        pc.create_index(index_name, dimension=768, metric="cosine",spec=spec)
    else:
        print("already there")

    index = pc.Index(index_name)
    from langchain.vectorstores import Pinecone
    vector_store = Pinecone.from_texts(chunks, embeddings, index_name=index_name)

    return vector_store




In [10]:
vector_store = extract_and_vectorize(pc, spec, embeddings)

already there


In [30]:

)

In [15]:
for txt in qa.stream({"query": "What is our problem statement?"}):
    print(txt['result'])

According to slide 2, the problem statement is "Instant Video Creation Through Text". This means taking a PDF brochure as input and converting it to a video, making a quiz, and showing analytics for both the video and quiz.According to slide 2, the problem statement is "Instant Video Creation Through Text". This means taking a PDF brochure as input and converting it to a video, making a quiz, and showing analytics for both the video and quiz.


In [22]:
#Truncate
index_name="bruno"
index=pc.Index(index_name)
index.delete(delete_all=True)

{}

In [11]:
from langchain.vectorstores import Pinecone
vector_store1 = Pinecone.from_existing_index(index_name="bruno", embedding=embeddings)

In [12]:
llm = ChatGroq(
    groq_api_key=groq_apikey,
    model_name="llama3-70b-8192",
    streaming=True,
   callbacks=[StreamingStdOutCallbackHandler()]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store1.as_retriever())